<a href="https://colab.research.google.com/github/shonyeajin/KCBD/blob/main/%EC%83%9D%EC%84%B1%EB%AA%A8%EB%8D%B8_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
  distribution=np.log(original_distribution)/ temperature
  distribution=np.exp(distribution)
  return distribution/np.sum(distribution)


In [4]:
import keras
import numpy as np
import tensorflow as tf

path=tf.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

614400/600901 [==============================] - 0s 0us/step


In [5]:
text=open(path).read().lower()
print('말뭉치 크기:', len(text))

말뭉치 크기: 600893


In [ ]:
print(text)

In [7]:
maxlen=60
step=3
sentences=[]
next_chars=[]

for i in range(0, len(text)-maxlen, step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])
print('시퀀스 개수:', len(sentences))

시퀀스 개수: 200278


In [8]:
chars=sorted(list(set(text)))
print('고유한 글자:', len(chars))

고유한 글자: 57


In [9]:
print(chars)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']


In [10]:
char_indices=dict((char,chars.index(char)) for char in chars)

In [11]:
print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}


In [12]:
#벡터화
x=np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y=np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]]=1
    y[i, char_indices[next_chars[i]]]=1
    


In [13]:
print(x)

[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False  True False ... False False False]]

 [[False False False ... False False False]
  [ True False False ... False False False]
  [ True False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False Fal

In [14]:
x.shape

(200278, 60, 57)

In [15]:
y.shape

(200278, 57)

In [19]:
from keras import layers

model=keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer=tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [22]:
def sample(preds, temperature=1.0):
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds)/temperature
  exp_preds=np.exp(preds)
  preds=exp_preds/np.sum(exp_preds)
  probas=np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [23]:
from keras.engine.base_layer_utils import generate_placeholders_from_shape
import random
import sys

random.seed(42)
start_index=random.randint(0, len(text) - maxlen -1)

for epoch in range(1, 60):
  print('에포크',epoch)
  model.fit(x, y, batch_size=128, epochs=1)

  seed_text=text[start_index: start_index + maxlen]
  print('--- 시드 텍스트: "' + seed_text + '"')
  
  for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ 온도:', temperature)
    generated_text=seed_text
    sys.stdout.write(generated_text)

    for i in range(400):
      sampled=np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
        sampled[0,t, char_indices[char]]=1.

      preds=model.predict(sampled, verbose=0)[0]
      next_index=sample(preds, temperature)
      next_char=chars[next_index]

      generated_text +=next_char
      generated_text = generated_text[1:]

      sys.stdout.write(next_char)
      sys.stdout.flush()

    print()

에포크 1
1565/1565 [==============================] - 12s 7ms/step - loss: 1.6131
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the serve of the self the present thereby merely the may be the streng there is a histered of the most consider of the such and his own the good the belief the some some compression of the seriened there is a more and the serve and thereby merely the merely the series of the self the series of the series of the spirit of the self and the serve and the present the soul, and the are thereby some w
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for interesting existence and the delights and and the something of the cosse of the should be far the could his could and
merelys of the most and histere and the may will of the series of the disposite all tere of the and not be and feels that the last and a sout religions of the sympathy belief

KeyboardInterrupt: ignored